In [ ]:
import os
import trimesh
import numpy as np
import glob
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import kaolin as kal
import kaolin.ops.conversions as tfs
from kaolin.io.modelnet import ModelNet
from kaolin.ops.gcn import GraphConv
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
#Downloading dataset
directory = tf.keras.utils.get_file(
    "modelnet.zip",
    "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip",
    extract=True
)
directory = os.path.join(os.path.dirname(directory), "ModelNet10")

In [ ]:
#Data observation
data1 = trimesh.load(os.path.join(directory, "monitor/train/monitor_0005.off"))
data1.show()

In [ ]:
#Data observation
data2 = trimesh.load(os.path.join(directory, "sofa/train/sofa_0005.off"))
data2.show()

In [ ]:
#Data observation
data3 = trimesh.load(os.path.join(directory, "bathtub/train/bathtub_0005.off"))
data3.show()

In [ ]:
#Useful Parameters
data_path = os.path.join(os.path.dirname(directory), "ModelNet10")
obj_categories = ['monitor', 'sofa', 'bed','bathtub', 'table']
number_of_points = 2048
number_of_batch_size = 32
rate = 0.001
total_epochs = 20

In [ ]:
training_loader = DataLoader(ModelNet(data_path, 
                                      categories=obj_categories, 
                                      split='train'), 
                             batch_size=number_of_batch_size, 
                             shuffle=True)

In [ ]:
validating_loader = DataLoader(ModelNet(data_path, 
                                        categories=obj_categories, 
                                        split='test'), 
                               batch_size=number_of_batch_size)

In [ ]:
#Building the model
model = GraphConv(len(obj_categories), 5)
opt = torch.optim.Adam(model.parameters(), lr=rate)
entropy_loss = torch.nn.CrossEntropyLoss()

In [ ]:
#Training the model

for epoch in range(total_epochs):

    print('\nEpoch: {epoch}\n')

    training_loss = 0.
    training_accuracy = 0.

    model.train()

    for index, (value, dicts) in enumerate(tqdm(training_loader)):
        parti_category = dicts['category']
        prediction = model(value)
        loss = criterion(prediction, parti_category.view(-1))
        training_loss += loss.item()
        loss.backward()
        opt.step()
        opt.zero_grad()

        #Calculating accuracy
        lbl_prediction = torch.argmax(prediction, dim=1)
        training_accuracy += torch.mean((lbl_prediction == parti_category.view(-1)).float()).item()

    #Displying trainig loss and accuracy
    print('Training loss:', training_loss / len(training_loader))
    print('Training accuracy:', training_accuracy / len(training_loader))

    validation_loss = 0.
    validation_accuracy = 0.

    model.eval()

    with torch.no_grad():
        for index, (value, dicts) in enumerate(tqdm(validating_loader)):
            parti_category = dicts['category']
            prediction = model(value)
            loss = criterion(prediction, parti_category.view(-1))
            validation_loss += loss.item()

            #Calculating accuracy
            lbl_prediction = torch.argmax(prediction, dim=1)
            validation_accuracy += torch.mean((lbl_prediction == parti_category.view(-1)).float()).item()

    #Displaying validation loss and accuracy
    print('Validation loss:', validation_loss / len(validating_loader))
    print('Validation accuracy:', validation_accuracy / len(validating_loader))

In [ ]:
#Evaluating the trained model on test datasets

testing_loader = DataLoader(ModelNet(data_path, 
                                     categories=obj_categories, 
                                     split='test'), 
                            shuffle=True, 
                            batch_size=number_of_batch_size)

value, dicts = next(iter(testing_loader))
parti_category = dicts['category']
prediction = model(value)
lbl_prediction = torch.max(prediction, axis=1)[1]

In [ ]:
#Displaying results

testIndex = 0 #We can enter the test index from 0..<number_of_batch_size
lbl = obj_categories[parti_category[testIndex].item()]
pred = obj_categories[lbl_prediction[testIndex]]

fig = plt.figure()
sub_plot = fig.add_subplot(1, 1, 1, projection='3d')


for index, point in enumerate(pointclouds):
    color = 'g'
    if pred == lbl:
      color = 'g'
    else:
      color = 'r'
    sub_plot.scatter(point[:, 0], point[:, 1], point[:, 2], c=color, s=3)

sub_plot.set_title('Original Image: {0}\nPredicted Image: {1}'.format(lbl, pred))
sub_plot.axis('off')
plt.show()